In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
TEST_DATASET_PATH = '../../PR/test_dataset'
MODEL_PATH = '../../PR/model2.ptm'
TEST_FILES = os.listdir(TEST_DATASET_PATH)

print(f'[verify] Found {len(TEST_FILES)} Test Files\n')

X_verify_all = []

test_files_pb = tqdm(TEST_FILES,'[preproc.transform] Transforming Data...')

for file in test_files_pb:
    file_path = os.path.join(TEST_DATASET_PATH, file)
    data = pd.read_csv(file_path)

    # 转换时间列，提取特征等
    data['time'] = pd.to_datetime(data['time'])
    data['hour'] = data['time'].dt.hour
    data['day_of_week'] = data['time'].dt.dayofweek
    data['month'] = data['time'].dt.month

    X_verify = data[['渔船ID', 'lat', 'lon', '速度', '方向', 'hour', 'day_of_week', 'month']]
    #X_test = data[['lat', 'lon', '速度', '方向', 'hour', 'day_of_week', 'month']]

    X_verify_all.append(X_verify)

# 将所有数据合并为一个大的 DataFrame
X_verify = pd.concat(X_verify_all, ignore_index=True)
X_verify

In [ ]:
X_verify_tensor = torch.tensor(X_verify[['lat', 'lon', '速度', '方向', 'hour', 'day_of_week', 'month']].values, dtype=torch.float32)
class TestDataset(Dataset):
    def __init__(self, features):
        self.features = features
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx]
    
verify_dataset = TestDataset(X_verify_tensor)
verify_loader = DataLoader(dataset=verify_dataset, batch_size=64, shuffle=False)

import torch.nn as nn
import torch.nn.functional as F

class FishingVesselNet(nn.Module):
    def __init__(self, num_features, num_classes):
        super(FishingVesselNet, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, num_classes)

    def forward(self, x):

        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = self.fc4(x)

        return x
    
# 实例化模型
num_features = X_verify.drop('渔船ID', axis=1).shape[1]
num_classes = 3

import torch.optim as optim
model = FishingVesselNet(num_features, num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
checkpoint = torch.load(MODEL_PATH)

model.eval()

In [ ]:
predictions = []
verify_loader_pb = tqdm(verify_loader, '[torch.test] Testing Accuracy')
with torch.no_grad():
    for inputs in verify_loader_pb:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())

In [ ]:
ids = X_verify['渔船ID'].values

# 创建一个DataFrame来存储预测结果
predictions_df = pd.DataFrame({
    '渔船ID': ids,
    'type': predictions
})

# 显示DataFrame的前几行以确认
predictions_df.head()

In [ ]:
X_replaced = predictions_df[['渔船ID', 'type']].replace({0:"刺网", 1:"围网", 2:"拖网"})
X_replaced

In [ ]:
X_final = X_replaced.drop_duplicates(subset='渔船ID', keep='first')
X_final

In [ ]:
X_final.to_csv('../../PR/submissions/nn/submission_nn.csv', index=False)